In [ ]:
!nvidia-smi

Sun Oct 24 15:16:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
train_rc = pd.read_csv('/content/drive/MyDrive/Tesi/train_rc.csv')
test_rc = pd.read_csv('/content/drive/MyDrive/Tesi/test_rc.csv')

In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 569072075 bytes (543 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
 59% 4096 Open              --
Path = /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
Type = zip
Physical Size = 569072075

  0%    
Would you like to replace the existing file:
  Path:     /content/dataset_bandPass2/111_1b3_Tc_sc_Meditron_2_bandPass2.wav
  Size:     110294 bytes (108 KiB)
  Modified: 2021-10-17 11:29:13
with the file from archive:
  Path:     dataset_bandPass2/111_1b3_Tc_sc_Meditron_2_bandPass2.wav
  Size:     110294 bytes (108 KiB)
  Modified: 2021-10

In [ ]:
print(train_rc.shape)
print(test_rc.shape)

(5518, 2)
(1380, 2)


In [ ]:
print(train_rc['label'].unique())

['none' 'wheeze' 'crackle' 'both']


In [ ]:
train_4 = train_rc.copy()
train_4 = train_4.replace('none', 0)
train_4 = train_4.replace('crackle', 1)
train_4 = train_4.replace('wheeze', 2)
train_4 = train_4.replace('both',3)
train_4['filename']="/content/dataset_bandPass2/"+train_4['filename']+'_bandPass2.wav'
train_4

,filename,label
0,/content/dataset_bandPass2/194_1b1_Pr_sc_Medit...,0
1,/content/dataset_bandPass2/113_1b1_Ar_sc_Litt3...,0
2,/content/dataset_bandPass2/211_1p2_Pr_mc_AKGC4...,0
3,/content/dataset_bandPass2/120_1b1_Lr_sc_Medit...,0
4,/content/dataset_bandPass2/203_1p4_Pr_mc_AKGC4...,0
...,...,...
5513,/content/dataset_bandPass2/158_1p4_Tc_mc_AKGC4...,0
5514,/content/dataset_bandPass2/154_1b3_Ll_mc_AKGC4...,1
5515,/content/dataset_bandPass2/113_1b1_Lr_sc_Litt3...,0
5516,/content/dataset_bandPass2/204_2b5_Al_mc_AKGC4...,2


In [ ]:
test_4 = test_rc.copy()
test_4 = test_4.replace('none', 0)
test_4 = test_4.replace('crackle', 1)
test_4 = test_4.replace('wheeze', 2)
test_4 = test_4.replace('both',3)
test_4['filename']="/content/dataset_bandPass2/"+test_4['filename']+'_bandPass2.wav'
test_4

,filename,label
0,/content/dataset_bandPass2/157_1b1_Pr_sc_Medit...,1
1,/content/dataset_bandPass2/130_2b2_Al_mc_AKGC4...,1
2,/content/dataset_bandPass2/210_1b1_Ar_sc_Medit...,0
3,/content/dataset_bandPass2/163_8b3_Ll_mc_AKGC4...,0
4,/content/dataset_bandPass2/178_1b6_Pl_mc_AKGC4...,0
...,...,...
1375,/content/dataset_bandPass2/147_1b2_Tc_mc_AKGC4...,0
1376,/content/dataset_bandPass2/192_2b2_Al_mc_LittC...,3
1377,/content/dataset_bandPass2/130_1p2_Pl_mc_AKGC4...,1
1378,/content/dataset_bandPass2/200_2p3_Pr_mc_AKGC4...,0


In [ ]:
train_4.groupby('label').count()

,filename
label,
0,2946
1,1469
2,704
3,399


In [ ]:
test_4.groupby('label').count()

,filename
label,
0,696
1,395
2,182
3,107


In [ ]:
import torchaudio
duration = 4
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result

In [ ]:
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    )
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
import torch
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Downloading: "https://github.com/s3prl/s3prl/archive/master.zip" to /root/.cache/torch/hub/master.zip


[s3prl.upstream.experts] Warning: can not import s3prl.upstream.roberta.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.hubert.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.vq_wav2vec.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.distiller.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.decoar2.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2_hug.expert: No module named 'transformers'. Pass.
[s3prl.hub] Warning: can not import s3prl.upstream.roberta.hubconf: No module nam

In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for aalbert classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.out_proj = nn.Linear(512, 4)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = features['last_hidden_state']
        res = torch.mean(features, dim=1)
        logit = self.classifier(res)
        return logit

In [ ]:
model = Classifier(m)

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime

def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+2}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/AudioAlbert/final_80-20_bandPass2_4-class_dropout=0.5_aalbert_lr={lr}_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

----------------------------

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=16
duration=4
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

Modello finale

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 4secondi tanh dropout=0.5 batch=8 aalbert FINAL

* Epoch 1/100


Loss: 1.63 #nan->0: 100%|██████████| 689/689 [05:25<00:00,  2.12it/s]


loss: 1.079994
acc: 0.568215
start validation


Loss: 0.86 #nan->0: 100%|██████████| 172/172 [01:17<00:00,  2.22it/s]


validation loss: 1.093532919883728
validation accuracy: 0.5661337209302325
Save new model!- 24/10 h17:24
--------------------
* Epoch 2/100


...

validation loss: 1.1025465726852417
validation accuracy: 0.6853197674418605
--------------------
* Epoch 9/100


Loss: 0.09 #nan->0: 100%|██████████| 689/689 [05:34<00:00,  2.06it/s]


loss: 0.205522
acc: 0.922533
start validation


Loss: 1.44 #nan->0: 100%|██████████| 172/172 [01:18<00:00,  2.20it/s]

validation loss: 1.1947723627090454
validation accuracy: 0.6933139534883721
--------------------


### Tentativi precedenti

-----------------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 1.09 #nan->0: 100%|██████████| 689/689 [04:13<00:00,  2.72it/s]


loss: 1.104897
acc: 0.536829
start validation


Loss: 1.00 #nan->0: 100%|██████████| 172/172 [00:53<00:00,  3.19it/s]


validation loss: 1.0352389812469482
validation accuracy: 0.5843023255813954
Save new model!- 21/10 h12:3
--------------------
* Epoch 2/100


...

validation loss: 1.2694287300109863
validation accuracy: 0.6598837209302325
--------------------
* Epoch 9/100


Loss: 0.04 #nan->0: 100%|██████████| 689/689 [04:13<00:00,  2.71it/s]


loss: 0.195824
acc: 0.927250
start validation


Loss: 2.65 #nan->0: 100%|██████████| 172/172 [00:53<00:00,  3.23it/s]

validation loss: 1.3511735200881958
validation accuracy: 0.6715116279069767
--------------------


------------------------------------

-------------------------------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 1.39 #nan->0: 100%|██████████| 689/689 [05:09<00:00,  2.23it/s]


loss: 1.199485
acc: 0.496734
start validation


Loss: 1.10 #nan->0: 100%|██████████| 172/172 [00:31<00:00,  5.44it/s]


validation loss: 1.1315791606903076
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.9530461430549622
validation accuracy: 0.6438953488372093
--------------------
* Epoch 17/100


Loss: 0.49 #nan->0: 100%|██████████| 689/689 [04:17<00:00,  2.68it/s]


loss: 0.618635
acc: 0.762337
start validation


Loss: 0.98 #nan->0: 100%|██████████| 172/172 [00:31<00:00,  5.43it/s]

validation loss: 0.9578137993812561
validation accuracy: 0.6635174418604651
--------------------


---------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.87 #nan->0: 100%|██████████| 689/689 [04:17<00:00,  2.68it/s]


loss: 1.125733
acc: 0.537192
start validation


Loss: 1.01 #nan->0: 100%|██████████| 172/172 [00:31<00:00,  5.50it/s]


validation loss: 1.0407752990722656
validation accuracy: 0.5799418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.0168224573135376
validation accuracy: 0.6824127906976745
--------------------
* Epoch 8/100


Loss: 0.04 #nan->0: 100%|██████████| 689/689 [04:17<00:00,  2.68it/s]


loss: 0.294968
acc: 0.893687
start validation


Loss: 1.17 #nan->0: 100%|██████████| 172/172 [00:31<00:00,  5.38it/s]

validation loss: 1.1512744426727295
validation accuracy: 0.6795058139534884
--------------------


-----------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =8e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.92 #nan->0: 100%|██████████| 689/689 [05:12<00:00,  2.20it/s]


loss: 1.131977
acc: 0.537192
start validation


Loss: 1.20 #nan->0: 100%|██████████| 172/172 [00:40<00:00,  4.28it/s]


validation loss: 1.0963331460952759
validation accuracy: 0.5428779069767442
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.103066325187683
validation accuracy: 0.6729651162790697
--------------------
* Epoch 9/100


Loss: 0.52 #nan->0: 100%|██████████| 689/689 [04:20<00:00,  2.65it/s]


loss: 0.306210
acc: 0.891328
start validation


Loss: 1.66 #nan->0: 100%|██████████| 172/172 [00:40<00:00,  4.23it/s]

validation loss: 1.1806399822235107
validation accuracy: 0.6765988372093024
--------------------


#### Audio Albert 4-class classification - 4MHz

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#train da capo a 4 MHz. lr=1e-4

* Epoch 1/100


Loss: 1.30 #nan->0: 100%|██████████| 689/689 [01:53<00:00,  6.04it/s]


loss: 1.147412
acc: 0.514695
start validation


Loss: 1.07 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.51it/s]


validation loss: 1.0755188465118408
validation accuracy: 0.561046511627907
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.2715373039245605
validation accuracy: 0.6068313953488372
--------------------
* Epoch 8/100


Loss: 0.05 #nan->0: 100%|██████████| 689/689 [01:01<00:00, 11.17it/s]


loss: 0.299772
acc: 0.888244
start validation


Loss: 1.14 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 14.21it/s]

validation loss: 1.3323147296905518
validation accuracy: 0.6344476744186046
--------------------


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#train da capo a 4 MHz

* Epoch 1/100


Loss: 0.97 #nan->0: 100%|██████████| 689/689 [01:12<00:00,  9.45it/s]


loss: 1.208863
acc: 0.482221
start validation


Loss: 1.12 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.15it/s]


validation loss: 1.1192036867141724
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.9638561010360718
validation accuracy: 0.6286337209302325
--------------------
* Epoch 15/100


Loss: 1.02 #nan->0: 100%|██████████| 689/689 [01:13<00:00,  9.38it/s]


loss: 0.755869
acc: 0.708454
start validation


Loss: 1.37 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.10it/s]

validation loss: 0.9435508251190186
validation accuracy: 0.6497093023255814
--------------------


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#train da capo a 3 MHz

* Epoch 1/100


Loss: 1.55 #nan->0: 100%|██████████| 689/689 [01:46<00:00,  6.45it/s]


loss: 1.201608
acc: 0.490566
start validation


Loss: 1.09 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.39it/s]


validation loss: 1.1258857250213623
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.9823052883148193
validation accuracy: 0.6082848837209303
--------------------
* Epoch 16/100


Loss: 0.30 #nan->0: 100%|██████████| 689/689 [00:53<00:00, 12.79it/s]


loss: 0.729697
acc: 0.715167
start validation


Loss: 1.19 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.47it/s]

validation loss: 0.9702529311180115
validation accuracy: 0.6206395348837209
--------------------


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#

* Epoch 1/100


Loss: 1.28 #nan->13: 100%|██████████| 689/689 [47:36<00:00,  4.15s/it]


loss: 1.364231
acc: 0.504438
start validation


Loss: 1.10 #nan->2: 100%|██████████| 172/172 [12:09<00:00,  4.24s/it]


validation loss: 1.2355105876922607
validation accuracy: 0.5154411764705882
Save new model!
--------------------
* Epoch 2/100


Loss: 0.83 #nan->13: 100%|██████████| 689/689 [48:21<00:00,  4.21s/it]


loss: 1.318933
acc: 0.524778
start validation


Loss: 1.11 #nan->2: 100%|██████████| 172/172 [11:42<00:00,  4.09s/it]


validation loss: 1.2214609384536743
validation accuracy: 0.5161764705882353
Save new model!
--------------------
* Epoch 3/100


Loss: 0.77 #nan->5:  44%|████▍     | 304/689 [21:12<19:08,  2.98s/it]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])

* Epoch 1/100


Loss: 1.14 #nan->13: 100%|██████████| 689/689 [37:13<00:00,  3.24s/it]


loss: 1.571352
acc: 0.335059
start validation


Loss: 1.20 #nan->2: 100%|██████████| 172/172 [08:57<00:00,  3.13s/it]


validation loss: 1.3547611236572266
validation accuracy: 0.5154411764705882
Save new model!
--------------------
* Epoch 2/100


Loss: 1.39 #nan->11:  94%|█████████▍| 650/689 [34:23<02:26,  3.76s/it]

In [ ]:
# ##PRIMO TRAINING
# from tqdm import tqdm
# import math
# from torch import autograd


# def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, max_stable=5):
#     best_val_loss = 1e9
#     counter = 0
#     nan=False
#     for epoch in range(epochs):
#         correct_predictions = 0
#         count=0
#         #tic = time()
#         print('* Epoch %d/%d' % (epoch+1, epochs))

#         avg_loss = 0
#         model.train()  # train mode
#         progress = tqdm(data_tr)
#         for batch in progress:
#             loss = 0
#             # data to device
#             X_batch, Y_batch = batch["representation"], batch["target"]
#             # print(X_batch.shape)
#             X_batch = X_batch.to(DEVICE)
#             Y_batch = Y_batch.to(DEVICE)
#             # with torch.autograd.detect_anomaly():
# #                     outputs = GB(inputs)
# #                     loss = LOSS(outputs, labels)
# # #                    loss.register_hook(lambda grad : print(grad))
# #                     loss.backward()
#             # set parameter gradients to zero
#             opt.zero_grad()
#             # forward
#             Y_pred = model(X_batch)
#             # print(X_batch)
#             # print(Y_pred)
#             # print(Y_batch)
#             # Y_pred = torch.tensor([e+1e-6 for e in Y_pred.reshape(1,16)[0].tolist()])
#             # Y_pred = Y_pred.reshape(8,2)
#             _, preds = torch.max(Y_pred, dim=1)
#             # loss = loss_fn(outputs[0], targets)
#             # loss = outputs[0]
#             # print(preds)
#             # print(Y_batch)
#             preds = preds.to(DEVICE)
#             Y_pred = Y_pred.to(DEVICE)
            

#             # print('---------')
#             # print('Y_pred')
#             # print(Y_pred)
            
#             # print('preds')
#             # print(preds)
#             # print('X_batch')
#             # print(X_batch)
#             # print('Y_batch')
#             # print(Y_batch)
#             # print('---------')

#             loss = loss_fn(Y_pred, Y_batch)# forward-pass
#             nan = math.isnan(loss)

#             if nan:
#               count=count+1
#               nan=False
#               continue
#               # print('---------')
#               # print('X_batch')
#               # print(X_batch.isnan().any())
#               # print('Y_pred')
#               # print(Y_pred)
#               # print(Y_pred.isnan().any())
              
#               # print('preds')
#               # print(preds.isnan().any())
              
#               # print('Y_batch')
#               # print(Y_batch.isnan().any())
#               # 
#               # print('---------')
#             # if math.isnan(loss):
#             #   print('ok')
#             # nan = True if loss is np.nan else False
#             # print('-------')
#             # print(loss)
#             # print('-------')
#             correct_predictions += torch.sum(preds == Y_batch)
#             progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
#             loss.backward()  # backward-pass
#             # torch.nn.utils.clip_grad_norm_(model.parameters(), 25)
#             opt.step()  # update weights
#             if not scheduler is None:
#                 scheduler.step()
#             # calculate loss to show the user
#             avg_loss += loss / len(data_tr)
#       #  toc = time()
#         avg_acc = correct_predictions.double() / (len(train_ds)-(batch_size*count))
        
#         print('loss: %f' % avg_loss)
#         print('acc: %f' % avg_acc)
#         # show intermediate results
#         model.eval()  # testing mode
#         val_loss = 0
#         print("start validation")
#         for v_b in tqdm(data_val):
#             X_val, Y_val = v_b["representation"], v_b["target"]
#             Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
#             val_loss += loss_fn(Y_hat, Y_val)
#         val_loss /= len(data_val)
#         print( f"validation loss: {val_loss}")
#         if val_loss <= best_val_loss and val_loss > 0:
#             counter = 0
#             print("Save new model!")
#             best_val_loss = val_loss
#             torch.save(model.state_dict(), '/content/drive/MyDrive/Tesi/AudioAlbert/wheeze_aalbert.h5')
#         else:
#             counter += 1
#         if counter == max_stable:
#             break